## TORONTO PART 3 : "Explore and cluster the neighborhoods in Toronto"

### 1. Import all libraries

In [77]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Download and Explore Our Toronto Data

In [78]:
# read from csv:
df_toronto = pd.read_csv("Toronto_part2.csv")
df_toronto.head()
print(df_toronto.shape)

(45, 6)


Explore how many Borough and Neighbourhood we have:

In [79]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 45 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto.

In [80]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with neighborhoods superimposed on top.

In [105]:
map_toronto= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Explore Neighborhoods in Toronto

#### Utilizing the Foursquare API to explore the neighborhoods and segment them

In [82]:
CLIENT_ID = 'MY' # your Foursquare ID
CLIENT_SECRET = 'MY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [83]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### For each Neighbourhood, let's get the top 100 venues that are within a radius of 500 meters

In [84]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500
toronto_venues=getNearbyVenues(names=df_toronto.Neighbourhood, latitudes=df_toronto.Latitude,
                    longitudes=df_toronto.Longitude)

Woburn
Cedarbrae
Scarborough Village
Agincourt
Upper Rouge
Hillcrest Village
Bayview Village
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Rosedale
Church and Wellesley
St. James Town
Berczy Park
Central Bay Street
Roselawn
Stn A PO Boxes 25 The Esplanade
Glencairn
Humewood-Cedarvale
Caledonia-Fairbanks
Christie
Queen's Park
Canada Post Gateway Processing Centre
Business reply mail Processing Centre969 Eastern
Islington Avenue
Humber Summit
Weston
Westmount
Northwest


### Let's check the shape of  a new dataset.

In [85]:
print(toronto_venues.shape)
print(toronto_venues.head())
print('The dataframe has  {} neighborhoods.'.format(
        len(toronto_venues.Neighborhood.unique())
    )
)

(803, 7)
  Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0       Woburn              43.770992              -79.216917   
1       Woburn              43.770992              -79.216917   
2       Woburn              43.770992              -79.216917   
3       Woburn              43.770992              -79.216917   
4    Cedarbrae              43.773136              -79.239476   

                   Venue  Venue Latitude  Venue Longitude     Venue Category  
0              Starbucks       43.770037       -79.221156        Coffee Shop  
1            Tim Hortons       43.770827       -79.223078        Coffee Shop  
2     Korean Grill House       43.770812       -79.214502  Korean Restaurant  
3  Jessies Variety Store       43.772778       -79.222500  Convenience Store  
4    Federick Restaurant       43.774697       -79.241142   Hakka Restaurant  
The dataframe has  43 neighborhoods.


### We found that two Neighborhoods do not have any venues. Let's get which Neighborhoods do not have any venues:

In [86]:
pd.concat([df_toronto.Neighbourhood,toronto_venues.Neighborhood]).drop_duplicates(keep=False)

4          Upper Rouge
40    Islington Avenue
dtype: object

#### Upper Rouge and Islington Avenue Neighbourhoods do not have any venues from Foresquare API.  Lets create new data frame which does not containe  Upper Rouge and Islington Avenue.

In [116]:
df_toronto_clean = df_toronto[df_toronto.Neighbourhood != 'Upper Rouge']
df_toronto_clean = df_toronto_clean[df_toronto_clean.Neighbourhood != 'Islington Avenue']
df_toronto_clean.head()

,Unnamed: 0,Postcode,Neighbourhood,Borough,Latitude,Longitude
0,0,M1G,Woburn,Scarborough,43.770992,-79.216917
1,1,M1H,Cedarbrae,Scarborough,43.773136,-79.239476
2,2,M1J,Scarborough Village,Scarborough,43.744734,-79.239476
3,3,M1S,Agincourt,Scarborough,43.794200,-79.262029
5,5,M2H,Hillcrest Village,North York,43.803762,-79.363452


In [88]:
df_toronto_clean.shape

(43, 6)

### Let's check how many venues were returned for each neighborhood

In [91]:
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4
Berczy Park,54,54,54,54,54,54
Business reply mail Processing Centre969 Eastern,17,17,17,17,17,17
Caledonia-Fairbanks,6,6,6,6,6,6


### Let's find out how many unique categories can be curated from all the returned venues

In [92]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 186 uniques categories.


## Analyze Each Neighborhood

In [93]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = toronto_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))

toronto_onehot = toronto_onehot.reindex(columns= cols)
toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hawaiian Restaurant,Health & Beauty Service,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Office,Optical Shop,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store,Yoga Studio
0,Woburn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Woburn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Woburn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Woburn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [94]:
# the new dataframe size.
toronto_onehot.shape

(803, 186)

###  let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [95]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hawaiian Restaurant,Health & Beauty Service,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Office,Optical Shop,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.25,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.25,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.25,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.250000,0.0,0.000000,0.25,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000

In [96]:
#the new size:
toronto_grouped.shape

(43, 186)

### print each neighborhood along with the top 5 most common venues

In [97]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
               venue  freq
0             Lounge  0.25
1     Breakfast Spot  0.25
2       Skating Rink  0.25
3     Clothing Store  0.25
4  Accessories Store  0.00


----Bayview Village----
                 venue  freq
0                 Café  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4            Nightclub  0.00


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.06
2          Café  0.04
3    Steakhouse  0.04
4   Cheese Shop  0.04


----Business reply mail Processing Centre969 Eastern----
                venue  freq
0         Yoga Studio  0.06
1  Light Rail Station  0.06
2                Park  0.06
3          Comic Shop  0.06
4    Recording Studio  0.06


----Caledonia-Fairbanks----
                  venue  freq
0                  Park  0.33
1  Fast Food Restaurant  0.17
2         Women's Store  0.17
3              Pharmacy  0.17
4                Market  0.17


----Canada Post Gateway Pro

               venue  freq
0               Park   0.5
1               Bank   0.5
2  Accessories Store   0.0
3        Music Venue   0.0
4        Men's Store   0.0




### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [98]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [99]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)


print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(43, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Dim Sum Restaurant,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop,Dog Run
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Bank,Café,Yoga Studio,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Drugstore
2,Berczy Park,Coffee Shop,Cocktail Bar,Café,Farmers Market,Seafood Restaurant,Restaurant,Steakhouse,Bakery,Cheese Shop,Beer Bar
3,Business reply mail Processing Centre969 Eastern,Yoga Studio,Fast Food Restaurant,Pizza Place,Butcher,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Light Rail Station
4,Caledonia-Fairbanks,Park,Pharmacy,Fast Food Restaurant,Market,Women's Store,Construction & Landscaping,Convenience Store,Electronics Store,Drugstore,Donut Shop


## Cluster Neighborhoods

In [119]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 3, 3, 0, 3, 3, 3, 3, 3], dtype=int32)

In [118]:
df_toronto_clean=df_toronto_clean.loc[:, ~df_toronto_clean.columns.str.contains('^Unnamed')]
df_toronto_clean.columns = ["Postcode","Neighborhood","Borough","Latitude","Longitude"]
df_toronto_clean.head()

,Postcode,Neighborhood,Borough,Latitude,Longitude
0,M1G,Woburn,Scarborough,43.770992,-79.216917
1,M1H,Cedarbrae,Scarborough,43.773136,-79.239476
2,M1J,Scarborough Village,Scarborough,43.744734,-79.239476
3,M1S,Agincourt,Scarborough,43.794200,-79.262029
5,M2H,Hillcrest Village,North York,43.803762,-79.363452


In [120]:
toronto_merged = df_toronto_clean

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1G,Woburn,Scarborough,43.770992,-79.216917,0,Coffee Shop,Convenience Store,Korean Restaurant,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop
1,M1H,Cedarbrae,Scarborough,43.773136,-79.239476,3,Hakka Restaurant,Athletics & Sports,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Caribbean Restaurant,Lounge,Deli / Bodega,Diner
2,M1J,Scarborough Village,Scarborough,43.744734,-79.239476,3,Women's Store,Playground,Department Store,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop,Dog Run,Discount Store,Diner
3,M1S,Agincourt,Scarborough,43.794200,-79.262029,3,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Dim Sum Restaurant,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop,Dog Run
5,M2H,Hillcrest Village,North York,43.803762,-79.363452,0,Mediterranean Restaurant,Fast Food Restaurant,Golf Course,Pool,Dog Run,Department Store,Electronics Store,Drugstore,Donut Shop,Discount Store


## visualize the resulting clusters

In [121]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

## CLuster 1

In [122]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Woburn,0,Coffee Shop,Convenience Store,Korean Restaurant,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop
5,Hillcrest Village,0,Mediterranean Restaurant,Fast Food Restaurant,Golf Course,Pool,Dog Run,Department Store,Electronics Store,Drugstore,Donut Shop,Discount Store
12,Downsview West,0,Grocery Store,Shopping Mall,Bank,Yoga Studio,Dim Sum Restaurant,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop,Dog Run
22,Lawrence Park,0,Bus Line,Park,Swim School,Dim Sum Restaurant,Dessert Shop,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop,Dog Run
24,North Toronto West,0,Coffee Shop,Sporting Goods Shop,Clothing Store,Yoga Studio,Chinese Restaurant,Fast Food Restaurant,Mexican Restaurant,Spa,Salon / Barbershop,Sandwich Place
25,Davisville,0,Dessert Shop,Sandwich Place,Pizza Place,Seafood Restaurant,Café,Coffee Shop,Sushi Restaurant,Italian Restaurant,Burger Joint,Farmers Market
26,Rosedale,0,Park,Playground,Trail,Yoga Studio,Department Store,Electronics Store,Drugstore,Donut Shop,Dog Run,Discount Store
27,Church and Wellesley,0,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Burger Joint,Restaurant,Gastropub,Mediterranean Restaurant,Men's Store,Fast Food Restaurant
36,Christie,0,Grocery Store,Café,Park,Convenience Store,Athletics & Sports,Diner,Italian Restaurant,Restaurant,Baby Store,Coffee Shop
42,Weston,0,Convenience Store,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop,Dog Run,Discount Store


## This cluster has very convinient neighboorhoods: many food venues, shopping malls. Defenetly if you like shopping this cluster is for you. 

## Cluster 2

In [123]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,Canada Post Gateway Processing Centre,1,Coffee Shop,Hotel,Fried Chicken Joint,Gym / Fitness Center,Mediterranean Restaurant,American Restaurant,Sandwich Place,Middle Eastern Restaurant,Burrito Place,Yoga Studio


## Cluster 3

In [124]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,The Beaches,2,Gym / Fitness Center,Coffee Shop,Other Great Outdoors,Pub,Dessert Shop,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop,Dog Run
28,St. James Town,2,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Gastropub,Italian Restaurant,Cocktail Bar,Bakery,Cosmetics Shop
44,Northwest,2,Drugstore,Bar,Rental Car Location,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Donut Shop,Dog Run


### This cluster has only food venues and some gymes. No parks, so if you like to go out outside in the park this area is not for you.

## Cluster 4

In [125]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Cedarbrae,3,Hakka Restaurant,Athletics & Sports,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Caribbean Restaurant,Lounge,Deli / Bodega,Diner
2,Scarborough Village,3,Women's Store,Playground,Department Store,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop,Dog Run,Discount Store,Diner
3,Agincourt,3,Lounge,Skating Rink,Clothing Store,Breakfast Spot,Dim Sum Restaurant,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop,Dog Run
6,Bayview Village,3,Japanese Restaurant,Chinese Restaurant,Bank,Café,Yoga Studio,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Drugstore
7,Willowdale South,3,Restaurant,Ramen Restaurant,Sandwich Place,Coffee Shop,Café,Pizza Place,Shopping Mall,Korean Restaurant,Discount Store,Japanese Restaurant
8,York Mills West,3,Park,Bank,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop,Dog Run
9,Willowdale West,3,Pharmacy,Wine Bar,Pizza Place,Coffee Shop,Deli / Bodega,Electronics Store,Drugstore,Donut Shop,Dog Run,Discount Store
10,Parkwoods,3,Construction & Landscaping,Fast Food Restaurant,Park,Food & Drink Shop,Yoga Studio,Dim Sum Restaurant,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop
11,Don Mills North,3,Gym / Fitness Center,Basketball Court,Baseball Field,Japanese Restaurant,Caribbean Restaurant,Café,Yoga Studio,Discount Store,Ethiopian Restaurant,Empanada Restaurant
13,Downsview Central,3,Korean Restaurant,Baseball Field,Food Truck,Yoga Studio,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Drugstore,Donut Shop


### This is the largest cluster. Interesting, that many places has the "Partk" as the most popular place. So many people in this areas has access to the park. They also has gyms, because gyms are in first or second the most popylar places.